In [45]:
#import the required Python packages

import pandas as pd
#import datetime   #check this
import numpy as np
import math
from pandas import DataFrame

math.exp = np.exp
math.pow = np.power
math.sqrt = np.sqrt

In [52]:
#df=pd.read_excel(r'Data/NWSAS_input_data.xlsx')
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
%autoreload
import nexus_tool

[autoreload of nexus_tool failed: Traceback (most recent call last):
  File "C:\Users\almulla\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\almulla\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\almulla\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 387, in update_generic
    update(a, b)
  File "C:\Users\almulla\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 342, in update_class
    if update_generic(old_obj, new_obj): continue
  File "C:\Users\almulla\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 387, in update_generic
    update(a, b)
  File "C:\Users\almulla\AppData\Local\Continuum\anaconda3\lib\site-packages\IP

In [6]:
nexus_tool.get_sw_tdh

# This notebook illustrates the main equations used for surface water pumping

**Source**: *The mathematics of pumping water*

This source is a manual for certain calculations which should be backed up with academic literature

# Step1: Calculate the head

Htot = Hstat + Hd + (Prt - Pres)

where: 

Htot: Total head

Hstat: Static head (It can be assumed in this case as the elevation difference)

Hd: Dynamic Head

Prt: Pressure on the surface of water in the receiving  tank.

Pres: Pressure on the surface of water in the reservoir.


## How to calculate the hydraulic head

Darct Weisbach Equation is used to calculate the hydraulic head

Hd = (K*V2)/(2*g)

Where: 

K: loss coefficient
V: velocity in the pipe in (m/sec)
g: Acceleration due to gravity=9.81 (m/sec2)

K = K_fitting + K_pipe

K_fitting: Depends on fitting items * number of items used. (see table in the source)

In long pipelines these local head losses (K_fitting) are often minor in comparison with energy losses due to friction and may be neglected.

so the main losses are due to friction

K_pipe = (f*l)/D 

f: friction coofcient 
L: lenght of the pipe in m
D: Diameter in m. 

f = 0.25/[log{(K/3.7*D)+(5.74/Re**0.9)}]**2

Where: 
K: Roughness factor (m): is a standard value obtained from standard tables and based upon the material of the pipe. 
Re: Reynolds number Re=(V*D)/v  
v: is the kinematic viscosity (m2/sec)

This equation can be simplified for laminar flow as follows: 
f = 64/Re



# Step2: Calculate the water flow

In this case the water flow will be calculated using WEAP. It will represent the flow in (m3/sec) and will be calculated for every pipeline.  



In [18]:
## INPUTS

#H_static = df['elevation']  #this is a gis layer
H_static =10

#L = df['pipe_lenght']      #Lenght of the pipeline segment in m
L=5

#D = df['diameter']        #this will be assumed for each pipeline segment
D=4


g = 9.81               #gravitational acceleration in m/sec2

Ken_visc=1.004**-6          #Kinematic Viscosity in (m2/s) value taken from https://www.engineersedge.com/physics/water__density_viscosity_specific_weight_13146.htm



Q=30                #assumption for testing

In [34]:
pi=3.14
A = (pi*D**2)/4
V = Q/A

dens = 1000 #density of water = 1000 kg/m3
Re = (V*D)/Ken_visc
pump_eff = 0.6
k=0.26  #roughness for cast iron https://www.engineersedge.com/fluid_flow/pipe-roughness.htm 

f =0.25/(np.log((k/(3.7*D))+(5.74/(Re*0.9)))**2)
#k = roughness factor k is a standard value obtained from standard tables and is based opon the material of the pipe, including any internal coatings and the internal condition of the pipeline i.e. good, normal or poor. 
#f=64/Re

#K_pipe = (f*L)/D



#K_loss = Kfit_tot + K_pipe  #Loss coefficient 

#H_dynamic = (K_loss*V**2)/(2*g)

#OR

H_dynamic = ((f*L*16*Q**2)/((D**5)*2*g*(pi**2)))


H_total = H_static + H_dynamic

In [35]:
f
#H_total

1.5507768212000188

In [36]:
H_dynamic

0.5636682097206777

# Step3: Calculate energy demand for surface water pumping

P = (Q * H * g * dens)/(pump_eff)

Where: 
P: is the energy requirement in (watts)
Q: water flow rate in (m3/sec)
H: total head in (m)
dens: is the density of water = 1000 kg/m3



In [34]:
P = (Q * H_total * g * dens)/(pump_eff)

P/1000

6071.002127388333

In [ ]:
##ORIGINAL CODE


def get_gw_tdh(df, gw_depth, wdd, oap, pld, tdh_gw, interp_method = 'nearest'):
    df[tdh_gw] = df[gw_depth] + wdd + oap + pld
    df[tdh_gw].replace(0, np.nan, inplace=True)
    # df[tdh_gw].interpolate(method = interp_method, axis=0, inplace=True)
    return df
    
def get_pumping_energy(df, trans_eff, pump_eff, pd_e, pwd, sswd, ed_e, tdh_gw, 
                       des_int, des_ener, desalination = False):
    Epump_plant_eff = trans_eff * pump_eff
    
    for i in range (1,13):
        _pd_e = '{}{}'.format(pd_e, i)
        _pwd = '{}{}'.format(pwd, i)
        _sswd = '{}{}'.format(sswd, i)
        _ed_e = '{}{}'.format(ed_e, i)
        
        df[_pd_e]=(9.81*(df[_pwd]/1000)*df[tdh_gw])/Epump_plant_eff
        df[_ed_e]=(df[_sswd]*df[tdh_gw]*0.00272)/Epump_plant_eff
        
        if desalination:
            df[_pd_e] += (df[_pwd]*df[des_int]*3600/1000)
            df[_ed_e] += (df['{}{}'.format(des_ener, i)]*1000000)
    return df
    
def get_annual_electricity(df, ed_e):
    df['annual_el_demand'] = df.filter(like=ed_e).sum(axis=1)
    return df

In [37]:
## TEST CODE FOR SURFACE WATER PUMPING

#Should define the following paramters: 
#L: length #paramter in the function
#D: Diameter #paramter in the function
#g: 9.81
#pi=3.14
#k=0.26 pipe roughness for cast iron 
#Ken_visc=1.004**-6          #Kinematic Viscosity in (m2/s) value taken from https://www.engineersedge.com/physics/water__density_viscosity_specific_weight_13146.htm
#H_static = df['elevation']  #this is a gis layer
H_static =10
V = Q/A
dens = 1000 #density of water = 1000 kg/m3

pump_eff = 0.6  #make sure that we have different naming for different pumps (surface water pump/ground water pump)


def get_gw_tdh(df, gw_depth, wdd, oap, pld, tdh_gw, interp_method = 'nearest'):
    df[tdh_gw] = df[gw_depth] + wdd + oap + pld
    df[tdh_gw].replace(0, np.nan, inplace=True)
    # df[tdh_gw].interpolate(method = interp_method, axis=0, inplace=True)
    return df

def Re(df,V,D,Ken_visc):
    Re = (V*D)/Ken_visc
    return df

def f(k,D,Re):
    f =0.25/(np.log((k/(3.7*D))+(5.74/(Re*0.9)))**2)
    return df

def get_sw_tdh(df, tdh_sw, elevation, f, L, w_flow, D, g, pi, interp_method = 'nearest'):
    df[tdh_sw] = df['elevation'] + ((f*L*16*(df[w_flow]**2))/((D**5)*2*g*(pi**2))) 
    df[tdh_sw].replace(0, np.nan, inplace=True)
    # df[tdh_sw].interpolate(method = interp_method, axis=0, inplace=True)
    return df


def get_pumping_energy(df, trans_eff, pump_eff, pd_e, pwd, sswd, ed_e, tdh_gw, tdh_sw 
                       des_int, des_ener, desalination = False):
    Epump_plant_eff = trans_eff * pump_eff
    
    for i in range (1,13):
        _pd_e = '{}{}'.format(pd_e, i)
        _pwd = '{}{}'.format(pwd, i)
        _sswd = '{}{}'.format(sswd, i)
        _ed_e = '{}{}'.format(ed_e, i)
        _w_flow = '{}{}'.format(w_flow, i) #this should be WEAP output
        
        df[_pd_e]=(9.81*(df[_pwd]/1000)*df[tdh_gw])/Epump_plant_eff
        df[_ed_e]=(df[_sswd]*df[tdh_gw]*0.00272)/Epump_plant_eff
        
        if desalination:
            df[_pd_e] += (df[_pwd]*df[des_int]*3600/1000)
            df[_ed_e] += (df['{}{}'.format(des_ener, i)]*1000000)
    return df
    
def get_annual_electricity(df, ed_e):
    df['annual_el_demand'] = df.filter(like=ed_e).sum(axis=1)
    return df

In [37]:
for i in range (1,13):
    df['PCWR_{}'.format(i)]=0       #PCWR: Peak Crop Water Requirement (l/s/ha) or "Duty", Previously PDWR
    df['PWD_{}'.format(i)]=0        #PWD: Peak Water Demand in (l/s)
    df['SSWD_{}'.format(i)]=0       #SSWD: Seasonal Scheme Water Demand in (m3)
    
#STEP 1: Compute the ACWR from ETc - check FAO1992- page 43-

#acwr=row['ETo_{}'.format(i)]*30*row['kc_{}'.format(i)] - row['eff_{}'.format(i)]*30 - row['awc']/12 ))
#once the available water content layer is obtained, the last past should be added to the equation

for crop in mode['Mode']:
    for i in range(1,13):
        eto = f'ETo_{i}'
        kc = f'kc_{i}'
        eff = f'eff_{i}'
        acwr = f'ACWR_{i}_'+crop
        pcwr = f'PCWR_{i}'
        pwd = f'PWD_{i}'
        sswd = f'SSWD_{i}'
        df[kc+'_'+crop] = float(mode.loc[mode['Mode']==crop,kc])
        ky=ky_dict[crop] #Yield response factor coeff = 0.8 for date palms, source TABLE 53-FAO: http://www.fao.org/3/y4360e/y4360e0b.htm 
        df[acwr] = (df[eto]*30*df[kc+'_'+crop]*ky - df[eff]*30 - (0.12*df[eff])*30) #Assumption: awc=12% effective rainfall
        df.loc[df[acwr]<0,acwr] = 0
        df[pcwr] += ((df[acwr]*10)/30)*2*0.012
        
        df[f'harvest_{i}_'+crop] = df['area_ha'] * np.array([x[crop] for x in df['Mode']]) * get_harvest_fraction(i,crop,'init','late')
        df[pwd] += (df[pcwr] *(df[f'harvest_{i}_'+crop]*24))/(pumping_hours_per_day*aeff*deff)
        df[sswd] += (df[acwr]*10*(df[f'harvest_{i}_'+crop])/(aeff*deff))

In [40]:
df.to_csv(output_file+'.csv',index=False)

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(output_file + '.xlsx', engine='xlsxwriter')

## Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='3crops')
summary.to_excel(writer, sheet_name='summary')

## Close the Pandas Excel writer and output the Excel file.
writer.save()